# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later
import sys
import os
import json
import pandas as pd
sys.path.insert(1, '../../../../src/')
import gt_modules.evaluation as evaluation
from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-movie3-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Production company explorative search") 

Consider the following exploratory information need:

> investigate the main companies working on cinema-related content. We want to know which are the main television production companies and the film production companies, which company distributes more film and some consideration about awards.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P106`     | profession   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q36479`    | The Lion King      | node |





Also consider

```
wd:Q36479 ?p ?obj .
```

is the BGP to retrieve all **properties of The Lion King**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for films

2. Identify the BGP for Netflix

3. Identify the BGP for television production company

4. Identify the BGP for film production company

5. Find the top-5 companies (amongst television production company and film production company) which produced the highest number of crime films (the result set must be a list of triples company IRI, label and #crime film).

6. Find the company (amongst television production company and film production company) which distributed more films (of any kind of film genre) that they did not produce (the result set must be a list of triples company IRI, label and #film).

7. Find how many company are listed in every stock exchange market (the result set must be a list of triples stock exchange IRI, label and #companies). 

8. Identify the BGP for Academy Award

9. Find the companies (among television and film production companies) that won at least 5 Academy Awards for Best Actress for the movies they produced (the result set must be a list of triples company IRI, label and #awards).

In [2]:
## startup the evaluation
# setup the file and create the empty json
ipname = "m3.ipynb"
pt = os.getcwd()+os.sep+ipname
evaluation.setup(pt)

The index of this workflow is: 2_6


## Task 1

In [3]:
# instance of The Lion King
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q36479 wdt:P31 ?obj .
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q29168811'), ('name', 'animated feature film')]
1


In [4]:
# super classes of animated feature film
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q29168811 wdt:P279 ?obj .
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q202866'), ('name', 'animated film')]
[('obj', 'http://www.wikidata.org/entity/Q24869'), ('name', 'feature film')]
2


In [5]:
# super class
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q24869 wdt:P279 ?obj .
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
1


In [6]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q11424"
og_name = "film"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"1", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 2_6
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_6.json
JSON object updated


`wd:Q11424` is the class of film

## Task 2

In [7]:
# properties of The Lion King
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q36479 ?prop ?obj .
    ?prop sc:name ?name.
    FILTER(!isLiteral(?obj)).
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('prop', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('prop', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('prop', 'http://www.wikidata.org/prop/direct/P1434'), ('name', 'takes place in fictional universe')]
[('prop', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('prop', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('prop', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('prop', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('prop', 'http://www.wikidata.org/prop/direct/P162'), ('name', 'producer')]
[('prop', 'http://www.wikidata.org/prop/direct/P1657'), ('name', 'MPAA film rating')]
[('prop', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('prop', 'http://www.wikidata.org/prop/direct/P175'), ('name', 'performer')]
[('prop', 

In [45]:
# use distributed by
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q36479 wdt:P750 ?obj .
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1323594'), ('name', 'Walt Disney Studios Motion Pictures')]
[('obj', 'http://www.wikidata.org/entity/Q135288'), ('name', 'Microsoft Store')]
[('obj', 'http://www.wikidata.org/entity/Q907311'), ('name', 'Netflix')]
[('obj', 'http://www.wikidata.org/entity/Q1097348'), ('name', 'InterCom')]
[('obj', 'http://www.wikidata.org/entity/Q54958752'), ('name', 'Disney+')]
5


In [8]:
### insert the result of TASK 2 in the file
og_uri = "http://www.wikidata.org/entity/Q907311"
og_name = "Netflix"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"2", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 2_6
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_6.json
JSON object updated


`wd:Q907311` is Netflix

## Task 3

In [9]:
# instance of Netflix
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q907311 wdt:P31 ?obj .
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4830453'), ('name', 'business')]
[('obj', 'http://www.wikidata.org/entity/Q723685'), ('name', 'video on demand')]
[('obj', 'http://www.wikidata.org/entity/Q891723'), ('name', 'public company')]
[('obj', 'http://www.wikidata.org/entity/Q841645'), ('name', 'Internet television')]
[('obj', 'http://www.wikidata.org/entity/Q15265344'), ('name', 'broadcaster')]
[('obj', 'http://www.wikidata.org/entity/Q368290'), ('name', 'film distributor')]
[('obj', 'http://www.wikidata.org/entity/Q10689397'), ('name', 'television production company')]
[('obj', 'http://www.wikidata.org/entity/Q1194970'), ('name', 'dot-com company')]
8


In [10]:
### insert the result of TASK 3 in the file
og_uri = "http://www.wikidata.org/entity/Q10689397"
og_name = "television production company"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"3", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 2_6
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_6.json
JSON object updated


`wd:Q10689397` is television production company

## Task 4

In [12]:
# superclass of television production company
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q10689397 wdt:P279 ?obj .
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1762059'), ('name', 'film production company')]
[('obj', 'http://www.wikidata.org/entity/Q19364326'), ('name', 'media studio facility')]
[('obj', 'http://www.wikidata.org/entity/Q11396960'), ('name', 'production company')]
3


In [13]:
### insert the result of TASK 4 in the file
og_uri = "http://www.wikidata.org/entity/Q1762059"
og_name = "film production company"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"4", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 2_6
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_6.json
JSON object updated


`wd:Q1762059` is film production company

## Task 5

In [14]:
# first find the crime genre
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    wd:Q36479 wdt:P136 ?obj .
    ?obj wdt:P31 ?inst.
    ?inst sc:name ?name.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q201658'), ('name', 'film genre')]
1


In [15]:
# first find the crime genre
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    ?obj wdt:P31 wd:Q201658.
    ?obj sc:name ?name.
    FILTER(REGEX(?name,"[C,c]rime")).
} 
ORDER BY ?name
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q959790'), ('name', 'crime film')]
[('obj', 'http://www.wikidata.org/entity/Q19367312'), ('name', 'crime thriller')]
[('obj', 'http://www.wikidata.org/entity/Q1788980'), ('name', 'crime-comedy film')]
3


In [16]:
# find how films are related with television or film production company
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    VALUES ?prod  {wd:Q1762059 wd:Q10689397}.
    ?film wdt:P31 wd:Q11424;
         ?prop ?comp.
    ?comp wdt:P31 ?prod.
    ?prop sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P123'), ('name', 'publisher')]
[('prop', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('prop', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('prop', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('prop', 'http://www.wikidata.org/prop/direct/P162'), ('name', 'producer')]
[('prop', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('prop', 'http://www.wikidata.org/prop/direct/P175'), ('name', 'performer')]
[('prop', 'http://www.wikidata.org/prop/direct/P264'), ('name', 'record label')]
[('prop', 'http://www.wikidata.org/prop/direct/P272'), ('name', 'production company')]
[('prop', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('prop', 'http://www.wikidata.org/prop/direct/P437'), ('name', 'distribution format')]
[('prop', 'http://www.wikidata.org/prop/direct/P449'), ('name', 'original broadcaster')]
[('prop', 'http:

In [18]:
#  amongst films of genre crime film
queryString = """
SELECT DISTINCT ?comp ?name COUNT(DISTINCT ?obj) AS ?count
WHERE { 
    VALUES ?prod  {wd:Q1762059 wd:Q10689397}.
    ?obj wdt:P31 wd:Q11424;
         wdt:P136 wd:Q959790;
         wdt:P272 ?comp.
    ?comp wdt:P31 ?prod.
    ?comp sc:name ?name.
} 
GROUP BY ?comp ?name
ORDER BY DESC (?count)
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('comp', 'http://www.wikidata.org/entity/Q126399'), ('name', 'Warner Bros.'), ('count', '157')]
[('comp', 'http://www.wikidata.org/entity/Q186941'), ('name', 'Columbia Pictures'), ('count', '117')]
[('comp', 'http://www.wikidata.org/entity/Q168383'), ('name', 'Universal Pictures'), ('count', '112')]
[('comp', 'http://www.wikidata.org/entity/Q179200'), ('name', 'Metro-Goldwyn-Mayer'), ('count', '111')]
[('comp', 'http://www.wikidata.org/entity/Q159846'), ('name', 'Paramount Pictures'), ('count', '108')]
5


In [19]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    val = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= val
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"5", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 2_6
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_6.json
JSON object updated


## Task 6

In [22]:
# films distributed but not produced
queryString = """
SELECT DISTINCT ?comp ?name COUNT(DISTINCT ?obj) AS ?count
WHERE { 
    VALUES ?prod  {wd:Q1762059 wd:Q10689397}.
    ?obj wdt:P31 wd:Q11424;
         wdt:P750 ?comp.
    FILTER NOT EXISTS{?obj wdt:P272 ?comp.}
    ?comp wdt:P31 ?prod.
    ?comp sc:name ?name.
} 
GROUP BY ?comp ?name
ORDER BY DESC (?count)
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('comp', 'http://www.wikidata.org/entity/Q907311'), ('name', 'Netflix'), ('count', '20568')]
[('comp', 'http://www.wikidata.org/entity/Q159846'), ('name', 'Paramount Pictures'), ('count', '2247')]
[('comp', 'http://www.wikidata.org/entity/Q186941'), ('name', 'Columbia Pictures'), ('count', '1549')]
[('comp', 'http://www.wikidata.org/entity/Q126399'), ('name', 'Warner Bros.'), ('count', '1463')]
[('comp', 'http://www.wikidata.org/entity/Q168383'), ('name', 'Universal Pictures'), ('count', '1435')]
5


In [23]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    val = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= val
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"6", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 2_6
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_6.json
JSON object updated


`wd:Q944287` is The Little Mermaid

## Task 7

In [24]:
# stock exchanges
queryString = """
SELECT ?x ?name COUNT(DISTINCT ?comp)
WHERE { 
    VALUES ?prod  {wd:Q1762059 wd:Q10689397}.
    ?comp wdt:P31 ?prod;
        wdt:P414 ?x.
    ?x sc:name ?name.
}
GROUP BY ?x ?name
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q732670'), ('name', 'Australian Securities Exchange'), ('callret-2', '1')]
[('x', 'http://www.wikidata.org/entity/Q82059'), ('name', 'NASDAQ'), ('callret-2', '3')]
[('x', 'http://www.wikidata.org/entity/Q13677'), ('name', 'New York Stock Exchange'), ('callret-2', '5')]
[('x', 'http://www.wikidata.org/entity/Q1661737'), ('name', 'Indonesia Stock Exchange'), ('callret-2', '1')]
[('x', 'http://www.wikidata.org/entity/Q217475'), ('name', 'Tokyo Stock Exchange'), ('callret-2', '2')]
[('x', 'http://www.wikidata.org/entity/Q496672'), ('name', 'Hong Kong Stock Exchange'), ('callret-2', '2')]
[('x', 'http://www.wikidata.org/entity/Q818723'), ('name', 'Toronto Stock Exchange'), ('callret-2', '1')]
7


In [25]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    val = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= val
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"7", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 2_6
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_6.json
JSON object updated


## Task 8

In [26]:
# object properties
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q36479 ?prop ?obj .
    ?prop sc:name ?name.
    FILTER(!isLiteral(?obj)).
}
ORDER BY ?name
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P2758'), ('name', 'CNC film rating (France)')]
[('prop', 'http://www.wikidata.org/prop/direct/P3402'), ('name', 'CNC film rating (Romania)')]
[('prop', 'http://www.wikidata.org/prop/direct/P2747'), ('name', 'Filmiroda rating')]
[('prop', 'http://www.wikidata.org/prop/direct/P1657'), ('name', 'MPAA film rating')]
[('prop', 'http://www.wikidata.org/prop/direct/P3834'), ('name', 'RTC film rating')]
[('prop', 'http://www.wikidata.org/prop/direct/P6942'), ('name', 'animator')]
[('prop', 'http://www.wikidata.org/prop/direct/P3174'), ('name', 'art director')]
[('prop', 'http://www.wikidata.org/prop/direct/P5021'), ('name', 'assessment')]
[('prop', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('prop', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('prop', 'http://www.wikidata.org/prop/direct/P674'), ('name', 'characters')]
[('prop', 'http://www.wikidata.org/prop/direct/P462'), ('name', 'co

In [27]:
# use award received
queryString = """
SELECT DISTINCT ?aw ?name
WHERE { 
    wd:Q36479 ?prop ?obj ;
        wdt:P166 ?aw.
    ?aw sc:name ?name.
}
ORDER BY ?name
"""

print("Results")
x=run_query(queryString)

Results
[('aw', 'http://www.wikidata.org/entity/Q488651'), ('name', 'Academy Award for Best Original Score')]
[('aw', 'http://www.wikidata.org/entity/Q823422'), ('name', 'National Film Registry')]
2


In [28]:
# find the Academy Awards class
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    wd:Q36479 ?prop ?obj ;
        wdt:P166 ?aw.
    ?aw wdt:P31 ?inst.
    ?inst sc:name ?name.
}
ORDER BY ?name
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
[('inst', 'http://www.wikidata.org/entity/Q96474685'), ('name', 'award for best original music')]
[('inst', 'http://www.wikidata.org/entity/Q861437'), ('name', 'cultural canon')]
[('inst', 'http://www.wikidata.org/entity/Q170584'), ('name', 'project')]
4


In [29]:
### insert the result of TASK 8 in the file
og_uri = "http://www.wikidata.org/entity/Q19020"
og_name = "Academy Awards"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"8", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 2_6
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_6.json
JSON object updated


`wd:Q19020` is the Academy Award

## Task 9

In [30]:
# find Academy Award for Best Actress
queryString = """
SELECT DISTINCT ?aw ?name
WHERE { 
    ?aw wdt:P31 wd:Q19020.
    ?aw sc:name ?name.
    filter(REGEX (?name,"Actress") ).
}
ORDER BY ?name
"""

print("Results")
x=run_query(queryString)

Results
[('aw', 'http://www.wikidata.org/entity/Q103618'), ('name', 'Academy Award for Best Actress')]
[('aw', 'http://www.wikidata.org/entity/Q106301'), ('name', 'Academy Award for Best Supporting Actress')]
2


In [31]:
# count the awards
queryString = """
SELECT ?comp ?name COUNT(DISTINCT ?film) AS ?awards
WHERE { 
    VALUES ?prod  {wd:Q1762059 wd:Q10689397}.
    ?film wdt:P31 wd:Q11424;
        wdt:P272 ?comp;
        wdt:P166 wd:Q103618.
    
    ?comp wdt:P31 ?prod.
    ?comp sc:name ?name.
}
GROUP BY ?comp ?name
HAVING (COUNT(DISTINCT ?film)>=5)
ORDER BY DESC (?awards)
"""

print("Results")
x=run_query(queryString)

Results
[('comp', 'http://www.wikidata.org/entity/Q159846'), ('name', 'Paramount Pictures'), ('awards', '11')]
[('comp', 'http://www.wikidata.org/entity/Q179200'), ('name', 'Metro-Goldwyn-Mayer'), ('awards', '11')]
[('comp', 'http://www.wikidata.org/entity/Q126399'), ('name', 'Warner Bros.'), ('awards', '9')]
[('comp', 'http://www.wikidata.org/entity/Q434841'), ('name', '20th Century Studios'), ('awards', '5')]
4


In [32]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    val = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= val
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"9", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 2_6
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_6.json
JSON object updated
